# Documentación del modelo de reputación bayesiano

Un modelo de reputación bayesiano es, en esencia, una distribución de probabilidades ya sea binomial o multinomial. En este proyecto se utilizaron 2 modelos de reputación:

## Modelo Binomial

Este tipo de modelo se utilizó para regular los ejercicios fáciles y difíciles de cada nivel de dificultad incorporado. Lo que se hace es tomar el desempeño del jugador tras completar la serie de ejercicios de una sesión de juego en concreto, asignarle un puntaje y enviar esta información al modelo. Utilizando los datos históricos de sus anteriores rondas junto con la nueva, el sistema calcula un nuevo % de ejercicios para la modalidad Fácil y Difícil.

## Modelo Multinomial

Este modelo se utilizó para determinar el % de ejercicios que se destinarán a cada objetivo de aprendizaje (Entiéndase por objetivo de aprendizaje la dificultad del ejercicio). Para determinar este porcentaje, este modelo se alimenta de los resultados del sistema binomial tras completar una sesión de juego.

## Imports

* __Flask__: Se utiliza principalmente para los métodos GET y POST de la API.
* __Sklearn__: Librería de ML. De esta se utilizan los StandardScaler y el Joblib para poder cargar la estructura de la red neuronal (modelo 1) con sus respectivos pesos.
* __Keras__: Modelos de red neuronal.
* __Tensorflow__: También hacen parte del modelo 1. Tensorflow se requiere porque a Keras se lo tragó Tensorflow, entonces no puede correr sin él. 
* __Graph, Model__: Graph y model son para cargar un modelo por defecto en caso de que la red neuronal entrenada no se haya podido cargar al momento de realizar la petición.
* __Pandas__: Se utiliza para operaciones de DataFrame.

In [ ]:
"""
Created on Wed Apr 17 19:41:58 2019

@author: Steven
"""

from flask import Flask, request, jsonify
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib 
import pandas as pd
import keras
from keras.models import load_model
import pickle
import sys
import tensorflow as tf
global graph,model
import numpy as np

Acá se configura la API y se cargan los valores predeterminados de la red neuronal y otros datos para funcionamiento

### Variables Importantes

__mediansDF__: Es un dataframe con las MEDIANAS de tiempo de lo que tardan los niños de los diferentes grados (1 a 5) de cada uno de los colegios que participaron en este proyecto
__med__: El mismo dataframe anterior, pero ahora en forma de GroupBy. Esto es para realizar un acceso más fácil a sus datos, pues de esta forma se podrá acceder como si fuese una matríz.

In [ ]:
# Flask
app = Flask(__name__)

# Load model
graph = tf.get_default_graph()
model = load_model('NewPlayerNeuralNet.h5')
sc = joblib.load('scaler.joblib')
mediansDF = pd.read_csv('medians.csv')
med = mediansDF.groupby(['colegio','loID','grado'])['tiempo'].mean()

Lo que se hará ahora es explicar todo el código en detalle y a fondo para que se pueda entender. No se explicará en el orden que aparecen las funciones, sino en orden de ejecución para que todo tenga sentido. Pero antes de eso hay que entender un poquito sobre cómo funcionan los sistemas de reputación. 

## Entendiendo el sistema de reputación

Previamente, en la introducción, se explicó brevemente lo que eran estos Sistemas de Reputación Bayesianos (BRS) y cómo se implementaron en el proyecto. Ahora, se explorará en detalle cómo funciona todo. Para una información más detallada y explicación teórica sobre esto, hay un libro bastante bueno llamado __Subjective Logic__ de __Audun Josang__, donde se explica a detalle y con ejemplos estos modelos. De igual manera se adjunta como ejemplo la tesis de maestría del profesor Ricardo Villanueva (https://www.slideshare.net/slideshow/embed_code/key/xptx6Mi4Q4VTR0), donde explica en la sección 7.2 cómo implementó el BRS en su proyecto. 

## Entiendiendo la solución propuesta

En esta sección se explorará la solución propuesta. El sistema de reputación por sí solo no puede hacer magia, pues lo único que hace, en esencia, es calcular una probabilidad de que algún valor pertenezca a una categoría u otra. Tomando el ejemplo de cómo opera el BRS del paper de ricardo, lo que se hace allí es redireccionar una petición por alguno de los servicios que estén disponibles. Como ya se tiene valores previos que indican qué tan bien reaccionan los servicios, lo que hace el sistema cuando se vuelve a calcular el % de "deseabilidad" preferir un servicio por encima de otro, es calcular la probabilidad de que la próxima vez que se utilice algún servicio, la experiencia tras utilizarlo sea buena. Es decir, que el servicio no presente ningún problema u error al usarse.

Pero cómo se define que el servicio fue bueno? Incialmente, se definieron métricas basadas para verificar cuándo a un servicio opera bien y así otorgarle un punto de reputación. Estos puntos de reputación son acumulables, así que entre más puntos tenga un servicio, (o en general, alguna opción o categoría del sistema), mejor y más deseable es utilizarlo. O dicho de otra forma, es más probable que algún valor pertenezca a esa categoría, o que esa categoría sea la preferente para elegir en algo. De la misma forma en este proyecto se deben definir métricas para categorizar los valores.

En este caso, lo que se busca es que el sistema de reputación pueda determinar qué tan bien o mal está un jugador en algún nivel de dificultad (Objetivo de aprendizaje). El sistema de reputación actuará para regular su transición a través de las categorías (dificultades) según como su desempeño vaya mejorando o empeorando, pero es responsabilidad del diseñador de la lógica definir cuándo o bajo qué criterios un niño se considera que está bien o mal. Esto, en esencia, sería como definir reglas de juego, o bien, métricas para medir su desempeño, las cuales serán las responsables de categorizar a un niño en algún nivel de dificultad.

Para entender mejor esto, considere la fórmula del BRS:

\begin{eqnarray*}
 categoría1 = \frac{categoría1+W*a}{W+(todas las categorías)}
\end{eqnarray*}

Si elimina las variables W\*a y W de la ecuación, podrá ver que la ecuación se simplifica de la siguiente forma:

\begin{eqnarray*}
 categoría1 = \frac{categoría1}{(todas las categorías)}
\end{eqnarray*}

Note que en esencia lo que esta fórmula hace es calcular una razón: ¿ Qué porcentaje de tiene **categoría1** con respecto a **todas las demás categorías** ? Lo que hacen W y W\*a es que el resultado no sea tan brusco. Por ejemplo:

Supongamos que estamos considerando un sistema de *Me gusta, No me gusta*. En este caso, tenemos un total de 2 categorías y cada una de ellas está almacenando la cantidad de veces que los usuarios han votado "me gusta" y "no me gusta". Asuma entonces la siguiente configuración:

* __Me gusta__: 10
* __No me gusta__: 0

Sin considerar las otras variables, tendríamos lo siguiente:

\begin{eqnarray*}
    Plikes = \frac{likes}{total}
    \\
    Plikes = \frac{likes}{likes+dislikes}
    \\
    Plikes=\frac{10}{10+0}
\end{eqnarray*}

La inclusión de las variables que maneja el sistema de reputación hace que no sea tan brusco el cambio. En vez de decir que directamente hay 0% de probabilidad que un valor pertenezca a una categoría sin votos, lo que hace es que se le asigna una probabilidad muy baja. Entonces, la fórmula para este sistema es la siguiente:

\begin{eqnarray*}
    Plikes = \frac{ likes + W*a }{W + (likes+dislikes) }
\end{eqnarray*}

Que de manera general sería

\begin{eqnarray*}
    Pcategoría_i = \frac{ categoría_i + W*a }{W + (todas-las-categorías) }
\end{eqnarray*}

Donde:

* __Pcategoría_i__: Es el porcentaje de reputación de esa categoría. Esto puede interpretarse como la probabilidad de que esta categoría sea preferida.
* __categoría_i__: La cantidad total de votos que tiene una categoría en específico.
* __todas las categorías__: Es la suma de los votos individuales de todas las categorías en consderación.
* __W__: Es una constante de distribución. Esta se utiliza para que cuando no hayan datos (Momento inicial), la distribución que tenga el modelo sea una uniforme. Está demostrado que esto puede lograrse con un W=2, por lo que el valor de esta constante **SIEMPRE DEBE SER 2** (el libro profundiza en esto)
* __a__: Es el valor incial. Hay 2 maneras de inicializar esta variable: Si no se tienen datos, lo que se hace es inciar todas las categorías en un mismo valor equitativo (Si se tienen 3 categorías, el valor de a será 1/3 para cada categoría); Si se tienen datos históricos, los valores de cada categoría serán esos valores que se tengan (Partiendo del ejemplo de los likes, suponga que se tomó una publicación aleatoria de facebook que contaba con 300 likes y 200 dislikes. Estos serían los valores para la variable a en cada categoría, que se denominan como "reputación dada por la comunidad" [También se profundiza en el libro]).

Entonces, el sistema de reputación solo se encargará de regular la cantidad de votos que tiene cada categoría para hacer una transición "smooth" entre ellas. Sin embargo, la lógica de cuándo se otorga reputación a una categoría en vez del a otra recae principalmente en quien quiera implementar el modelo. 

## Métricas para categorizar

Como el BRS por sí solo no hace nada, es necesario definir variables que permitan alimentar al modelo para tener noción de cuándo *se consider que* un jugador va bien o mal. Es necesario recolectar variables dentro del propio juego para analizar sus valores y determinar cuándo estos son buenos o malos y así traducir esto a puntos de reputación. Las variables seleccionadas para este proyecto fueron:

* __dificultad__: Cada nivel de dificultad tiene ejercicios categorizados como fáciles y difíciles. Su dificultad yace en los valores que estos toman. Por ejemplo, para la dificultad 1 que son sumas de un dígito + un dígito, los ejercicios fáciles son aquellos cuya respuesta es un dígito, mientras que los difíciles son aquellos cuya respuesta tiene dos dígitos.
* __correcto__: Si la pregunta fue contestada de manera correcta
* __Titubeo__: Conocido en el modelo 1 como *answerChangedCount*, indica cuántas veces se cambió entre opciones de respuesta antes de pulsar el botón de "enviar". Esto sirve como medida para ver qué tan seguro está el niño de su respuesta.
* __Tiempo__: El tiempo que demoró en contestar el ejercicio.

Analizando el valor de estas variables es que se determinará si un niño va bien o mal en un nivel de dificultad.

## Cómo se calcula todo?



In [ ]:
@app.route('/nextIntensity', methods=['POST'])
def onPerformanceReceived():
    playerData = request.get_json(force=True)    
    d = {'LOs': playerData['LO'],
         'grado': playerData['grado'],
         'sesion': playerData['sesion'],
         'binLO0': playerData['binLO0'],
         'binLO1': playerData['binLO1'],
         'binLO2': playerData['binLO2'],
         'binLO3': playerData['binLO3'],
         'binLO4': playerData['binLO4'],
         'binPer': playerData['binPer'],
         'mulLO0': playerData['mulLO0'],
         'mulLO1': playerData['mulLO1'],
         'mulLO2': playerData['mulLO2'],
         'mulLO3': playerData['mulLO3'],
         'mulLO4': playerData['mulLO4'],
         'mulPer': playerData['mulPer'],
         'tiempos': playerData['tiempos'],
         'titubeo': playerData['titubeo'],
         'isCorrect': playerData['correcto'],
         'colegio' : playerData['tipoEscuela'],
         'dificultades': playerData['dificultad']}
    
    n = len(d['isCorrect'])
    W = 2;
    bin_a = 0.5
    mult_a = 0.33
    bin_aging = 0.5
    mult_aging = 0.3
    #Variables
    sesion = d['sesion']
    grade = d['grado']
    isPublicOrPrivate = d['colegio']
    #Performance
    difficulty = d['dificultades']
    time = d['tiempos']
    ansChangedCount = d['titubeo']
    isCorrect = d['isCorrect']
    LOs = d['LOs']
    total, nEasy, nHard = obtainEasyAndHardCount(difficulty,LOs)
    #Binomial instances
    binomial_hist = [d['binLO0'],d['binLO1'],d['binLO2'],d['binLO3'],d['binLO4']]
    binomial_per = d['binPer']
    #Multinomial instances
    multinomial_hist = [d['mulLO0'],d['mulLO1'],d['mulLO2'],d['mulLO3'],d['mulLO4']]
    multinomial_per = d['mulPer']
    binomial_hist, binomial_per, multinomial_hist, multinomial_per, intensities = obtainNextIntensity(nEasy, nHard, total, sesion,
                        binomial_hist, binomial_per, bin_a, bin_aging, multinomial_hist, multinomial_per, mult_a, mult_aging,
                        W, difficulty, time, ansChangedCount, isCorrect, LOs, grade, isPublicOrPrivate)
    
    binJSON = "\"binLO0\":"+str(binomial_hist[0])+", \"binLO1\":"+str(binomial_hist[1])+", \"binLO2\":"+str(binomial_hist[2])+", \"binLO3\":"+str(binomial_hist[3])+", \"binLO4\":"+str(binomial_hist[4])+", \"binPer\":"+str(binomial_per)+", "
    mulJSON = "\"mulLO0\":"+str(multinomial_hist[0])+", \"mulLO1\":"+str(multinomial_hist[1])+", \"mulLO2\":"+str(multinomial_hist[2])+", \"mulLO3\":"+str(multinomial_hist[3])+", \"mulLO4\":"+str(multinomial_hist[4])+", \"mulPer\":"+str(multinomial_per)+", "
    intensityJSON = ""
    for i in range(0,5):
        intensityJSON = intensityJSON + "\"LOIN"+str(i)+"\":["+str(intensities[i])+","+str(binomial_per[i][0])+","+str(binomial_per[i][1])+"]"
        if(i<4):
            intensityJSON = intensityJSON + ", "
            
    jsonIntensityChild = "\"Intensities\": {"+intensityJSON+"}"
    return "{ "+binJSON+mulJSON+jsonIntensityChild+" }"

In [ ]:
def obtainEasyAndHardCount(difficulties, LO):
    nEasy = []
    nHard = []
    total = []
    for i in range (0,5):
        easyCount = 0
        hardCount = 0
        totalCount = 0
        for j in range(0,len(difficulties)):
            if(LO[j] == i):
                totalCount=totalCount+1
                if(difficulties[j] == 0):
                    easyCount=easyCount+1
                else:
                    hardCount=hardCount+1
        nEasy.append(easyCount)
        nHard.append(hardCount)
        total.append(totalCount)

    return [total, nEasy, nHard]

In [ ]:
def obtainNextIntensity(nEasy, nHard, total, nSesion, binHist, binPer, bin_a, binAging, mulHist, mulPer, mult_a, mulAging, W, difficulty, times, titubeo, isCorrect, LO, grade, isPublicOrPrivate):
    binomialEasyRatesPerLO = []
    #Calculating binomial scores
    if(nSesion % 5 == 0):
        for i in range (0,5):
            agedValues = applyBinomialAging(binHist[i],binAging)
            agedValues = agedValues.tolist()
            binHist[i] = agedValues
    
    n = 0
    for i in range (0,5):
        if(total[i]>0):
            easyExcercises, hardExcercises = nEasy[i], nHard[i]
            #Calculate scores
            scores = []
            diff = []
            for j in range(n,n+total[i]):
                scores.append(giveScoreToPlayer(difficulty[j],times[j],titubeo[j],isCorrect[j],LO[j],grade,isPublicOrPrivate))
                diff.append(difficulty[j])
            n=n+total[i]#Here goes calculus
            #Calculate scores
            binomialScores = rateRules1(total[i],easyExcercises,hardExcercises,scores,diff)
            binHist[i].append(binomialScores)
            binomialRates = binomialRate(binHist[i],W,bin_a)
            binPer[i] = binomialRates.tolist()
            print(binPer)
        binomialEasyRatesPerLO.append(binPer[i][0])

    #Calculating multinomial
    multinomialScores = rateRules2(binomialEasyRatesPerLO,total)
    for i in range(0,5):
        if(nSesion % 10 == 0):
            agedValues = applyMultinomialAging(mulHist[i],mulAging)
            agedValues = agedValues.tolist()
            mulHist[i] = agedValues
        mulHist[i].append(multinomialScores[i])
        mulPer[i] = multinomialRate(mulHist[i],W,mult_a).tolist()
    
    #Calculate intensities
    intensities = calculateIntensity(mulPer)
    return [binHist, binPer, mulHist, mulPer, intensities]

In [ ]:

#Apply aging factor to multinomial model
def applyMultinomialAging(multinomialHist,agingFactor):
    for i in range(0,len(multinomialHist)):
        multinomialHist[i][0] = multinomialHist[i][0]*agingFactor
        multinomialHist[i][1] = multinomialHist[i][1]*agingFactor
        multinomialHist[i][2] = multinomialHist[i][2]*agingFactor
    return np.around(multinomialHist,decimals=2)

#Apply aging factor to binomial model
def applyBinomialAging(binomialHist,agingFactor):
    for i in range(0,len(binomialHist)):
        binomialHist[i][0] = binomialHist[i][0]*agingFactor
        binomialHist[i][1] = binomialHist[i][1]*agingFactor
    return np.around(binomialHist,decimals=2)

In [ ]:
#Get score from player performance
def giveScoreToPlayer(difficulty, time, ansChangedCount, isCorrect, LO, grade, isPublicOrPrivate):
    medianTime = med[isPublicOrPrivate,LO,grade]+2
    if(time <= medianTime and isCorrect == 1):
        S = 20;
    else:
        time = time-medianTime;
        bonus = difficulty+1
        S = ((20*bonus)*(1-time/medianTime)-2*(ansChangedCount-1))*isCorrect
    return S

In [ ]:
#Sum binomial rates to get easy and hard %
def sumEasyAndHardScores(array):
    totalE = 0
    totalH = 0
    for i in range(0,len(array)):
        totalE = totalE+array[i][0]
        totalH = totalH+array[i][1]
    return [totalE,totalH]

#Sum bad, medium and hard categories rates to get %
def sumPerformanceCategories(array):
    totalB = 0
    totalM = 0
    totalG = 0
    print(len(array))
    for i in range(0,len(array)):
        totalB = totalB + array[i][0]
        totalM = totalM + array[i][1]
        totalG = totalG + array[i][2]
    return [totalB, totalM, totalG]

#General Binomial model
def binomialRate(binomialS, W, a):
    r1,r2 = sumEasyAndHardScores(binomialS)
    den = W+r1+r2
    S1 = (r1+W*a)/den
    S2 = (r2+W*a)/den
    scores = [S1,S2]
    scores = np.around(scores,decimals=2)
    return scores

#General Multinomial model
def multinomialRate(multinomialS, W, a):
    print(multinomialS)
    r1,r2,r3 = sumPerformanceCategories(multinomialS)
    den = W+r1+r2+r3
    S1 = (r1+W*a) / den
    S2 = (r2+W*a) / den
    S3 = (r3+W*a) / den
    scores = [S1,S2,S3]
    scores = np.around(scores,decimals=2)
    return scores

In [ ]:
#This returns the NEXT amount of easy and hard exercises
def rateRules1(nExercises, nEasy, nHard, scores, difficulty):
    easyCounter = nEasy;
    hardCounter = nHard;
    for i in range(0,len(scores)):
        if(difficulty[i] == 0):
            if(scores[i] >= 16):
                hardCounter=hardCounter+1
                easyCounter=easyCounter-1
            elif(scores[i] < 10 ):
                easyCounter=easyCounter+1
                hardCounter=hardCounter-1
                
            if(easyCounter < 0):
                easyCounter = 0
            elif(easyCounter > nExercises):
                easyCounter = nExercises
                
            if(hardCounter < 0):
                hardCounter = 0
            elif(hardCounter > nExercises):
                hardCounter = nExercises
        else:
            if(scores[i] >= 28):
                hardCounter=hardCounter+1
                easyCounter=easyCounter-1
            elif(scores[i] < 12 ):
                easyCounter=easyCounter+1
                hardCounter=hardCounter-1
                
            if(easyCounter < 0):
                easyCounter = 0
            elif(easyCounter > nExercises):
                easyCounter = nExercises
                
            if(hardCounter < 0):
                hardCounter = 0
            elif(hardCounter > nExercises):
                hardCounter = nExercises
            
    score = [easyCounter, hardCounter]
    return score;

#This increases the amount of % of exercises in each LO
def rateRules2(easyBinomialRates, exercises):
    multinomialCounter = [[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
    for i in range(0,5):
        if(exercises[i] > 0):
            if(easyBinomialRates[i] >= 0.7):
                multinomialCounter[i][0] = multinomialCounter[i][0]+2
                if( i == 0 ):
                    multinomialCounter[i+1][0] = multinomialCounter[i+1][0]+1
                j = i-1
                while(j >= 0):
                    multinomialCounter[j][1] = multinomialCounter[j][1]+1
                    j = j-1
            elif(easyBinomialRates[i] >= 0.4 and easyBinomialRates[i] < 0.7):
                multinomialCounter[i][1] = multinomialCounter[i][1]+1
            else:
                multinomialCounter[i][2] = multinomialCounter[i][2]+2
                if(i < 4):
                    multinomialCounter[i+1][1] = multinomialCounter[i+1][1]+1
                j = i-1
                while(j >= 0):
                    multinomialCounter[j][2] = multinomialCounter[j][2]+1
                    j = j-1
                    
    return multinomialCounter

In [ ]:
#Get next group of intensities
def calculateIntensity(multinomialPer):
    i = 0;
    perN = []
    while(i < 4 and multinomialPer[i][2] >= 0.8):
        perN.append(0)
        i = i+1;
           
    valuesN = []
    valuesN.append(1-multinomialPer[i][2])
    j = i+1
    k=1
    while(j < 5 and j <= i+2 and multinomialPer[j][0] <= 0.5):
       valuesN.append(1-multinomialPer[j][2])
       k=k+1
       j=j+1
       
    totalN = sum(valuesN)
    for j in range(0,len(valuesN)):
        perN.append(valuesN[j] / totalN)
        i=i+1
        
    for j in range(i,5):
        perN.append(0)
        
    return np.around(perN,decimals=2)

In [ ]:
@app.route('/onNewPlayer', methods=['POST'])
def predict():
    playerData = request.get_json(force=True)
    d = {'edad': [playerData['edad']], 'escuela':[playerData['tipoEscuela']], 'genero': [playerData['genero']], 'grado': [playerData['grado']]}
    df = pd.DataFrame(data=d)
    X = sc.transform(df)
    with graph.as_default():
        intensities = model.predict(X)
        if(d['grado'][0] <= 3):
            total = intensities[0,0]+intensities[0,1]+intensities[0,2]
            intensities[0,0] = intensities[0,0]/total
            intensities[0,1] = intensities[0,1]/total
            intensities[0,2] = intensities[0,2]/total
            intensities[0,3] = 0
            intensities[0,4] = 0
        LO0 = "\"LOIN0\":["+str(intensities[0,0])+",1,0],"
        LO1 = "\"LOIN1\":["+str(intensities[0,1])+",1,0],"
        LO2 = "\"LOIN2\":["+str(intensities[0,2])+",1,0],"
        LO3 = "\"LOIN3\":["+str(intensities[0,3])+",1,0],"
        LO4 = "\"LOIN4\":["+str(intensities[0,4])+",1,0]"
        jsonFormatIntensities = "{"+LO0+LO1+LO2+LO3+LO4+"}"
        return jsonFormatIntensities